In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as Data
import re
import numpy as np
import math

In [ ]:
import jieba

In [ ]:
with open('/content/drive/MyDrive/英文文档NLP/data.txt','r',encoding='utf-8') as file:
    content=file.readlines()


In [ ]:
def preprocessing(sent):
    pattern = re.compile(r'([\u4e00-\u9fa5])')
    chars = "".join(pattern.findall(sent))
    #chars =[w for w in chars if len(w) > 0]
    return chars

In [ ]:
def dic(content):
  #sentences=preprocessing(content)
  
  word_list=preprocessing(content)
  #word_list=jieba.lcut(word_list)
  word_list = ' '.join(word_list).split()
  word_list = list(set(word_list))

  word2id = {w: i for i, w in enumerate(word_list)}
  id2word = {i: w for i, w in enumerate(word_list)}
  return word2id

In [ ]:
with open('/content/drive/MyDrive/英文文档NLP/data.txt','r',encoding='utf-8') as file:
    content_total=file.read()
    word2id=dic(content_total)

In [ ]:
def computeCE(sen,word2id):

    
    
  #sentences = ['我 喜欢 狗', '我 讨厌 牛奶', '我 做 自然语言处理', '我 看 电影']
  sentences = sen
  
  #word_list = ' '.join(sentences).split()
  #word_list = list(set(word_list))

  #word2id = {w: i for i, w in enumerate(word_list)}
  #id2word = {i: w for i, w in enumerate(word_list)}

  def make_batch(sentence):
      input_batch = []
      target_batch = []
      #word = sentence.split()
      
      for sen in sentence:
        #word=jieba.lcut(sen)
        word=' '.join(sen).split()
        for n in range(len(word)-5):
          inputs = [word2id[w] for w in word[n:n+5]]
          target = word2id[word[n+5]]
          input_batch.append(inputs)
          target_batch.append(target)
      #print(input_batch,target_batch)
      return input_batch, target_batch

  input_batch, target_batch = make_batch(sentences)
  input_batch = torch.LongTensor(input_batch)
  target_batch = torch.LongTensor(target_batch)


  n_class = len(word2id) # 词表大小
  n_hidden = 10   # 隐藏层神经单元数目
  m = 5  # 词向量维度
  n_step = 5   # 代表n-gram模型 2就是2-gram 用前两个词预测最后一个词
  # 搭建模型
  class NNLM(nn.Module):
      def __init__(self):
          super(NNLM, self).__init__()
          self.embd = nn.Embedding(n_class, m)
          self.W = nn.Parameter(torch.randn(n_step*m, n_hidden))
          self.b = nn.Parameter(torch.randn(n_hidden))
          self.U = nn.Parameter(torch.randn(n_hidden, n_class))
          self.d = nn.Parameter(torch.randn(n_class))

      def forward(self, x):
          x = self.embd(x)
          x = x.view(-1, n_step*m)
          tanh = torch.tanh(torch.mm(x, self.W) + self.b)
          output = torch.mm(tanh, self.U) + self.d

          return output

  model = NNLM()  # 模型实例化

  criterion = nn.CrossEntropyLoss()  # 选择损失函数
  optimizer = optim.Adam(model.parameters(), lr=0.001)  # 选择优化器

  # 迭代训练
  for i in range(1000):
      optimizer.zero_grad()  # 梯度清零

      output = model(input_batch)
      
      loss = criterion(output, target_batch)
      loss.backward()
      optimizer.step()


  #predict = model(input_batch).data.max(1, keepdim=True)[1]
  #print([sen.split()[:-1] for sen in sentences], '-->', [id2word[n.item()] for n in predict.squeeze()])
      
  return loss,model

In [ ]:
ppltotal=0.
count=0
sen1=[]
for i in range(len(content)):
  sen=preprocessing(content[i])
  if len(sen)>5:
    sen1.append(sen)
train_data=sen1[0:20000]
    #if math.isnan(CE)==False:
      #ppltotal+=pow(2,CE)
      #count+=1
  #else:
    #break
    #pr
    
    #int(i)
#PPL=ppltotal/count

In [ ]:
CE,model=computeCE(train_data,word2id)

print(CE.item())

In [ ]:
def make_batch(sentence):
    input_batch = []
    target_batch = []
      #word = sentence.split()
      
    for sen in sentence:
        #word=jieba.lcut(sen)
      word=' '.join(sen).split()
      for n in range(len(word)-5):
        inputs = [word2id[w] for w in word[n:n+5]]
        target = word2id[word[n+5]]
        input_batch.append(inputs)
        target_batch.append(target)
      #print(input_batch,target_batch)
    return input_batch, target_batch

In [ ]:
testdata=sen1[20000:30000]
loss=[]
for test_data in testdata:
  input_batch, target_batch = make_batch(test_data)
  input_batch = torch.LongTensor(input_batch)
  target_batch = torch.LongTensor(target_batch)
  criterion = nn.CrossEntropyLoss()
  l=criterion(model(input_batch),target_batch)
  print(l)
  loss.append(l)

In [ ]:
testdata=sen1[20000:50000]
loss=[]

input_batch, target_batch = make_batch(testdata)
input_batch = torch.LongTensor(input_batch)
target_batch = torch.LongTensor(target_batch)
criterion = nn.CrossEntropyLoss()
l=criterion(model(input_batch),target_batch)
print(l)
loss.append(l)

In [ ]:
ppl=sum(np.exp(loss[i].item()) for i in range(len(loss)))

In [ ]:
ppl

In [ ]:
model.state_dict()

In [ ]:
torch.save(model.state_dict(),'/content/drive/MyDrive/英文文档NLP/NNLM.pkl' )

In [ ]:
computeCE(preprocessing(content[2]))+computeCE(preprocessing(content[0]))